In [ ]:
import scanpy as sc
import scanpy.external as sce
import os 
import pandas as pd 
import numpy as np
import seaborn as sns
from functions import compare_obs_values_within_groups_to_excel
import matplotlib.pyplot as plt
import liana as li
# from statannotations.Annotator import Annotator

adata_name='venous_ec'
figures = "data/figures/figures"
data = "data/single_cell_files/scanpy_files"

os.makedirs(figures, exist_ok=True)
sc.set_figure_params(dpi_save=300, fontsize=8, figsize=(2,2))
sc.settings.figdir = figures
sns.set_style('white', rc={
    'xtick.bottom': True,
    'ytick.left': True,
})
plt.rcParams["font.family"] = "Arial"
size=15

In [ ]:
adata = sc.read(f"{data}/venous_ec_vessel_size_velocity.gz.h5ad")
adata_mural=sc.read(f"{data}/venous_ec_mural_velocity.gz.h5ad")

adata.X = adata.layers['log1p'].copy()
del adata.layers
adata_mural.X = adata_mural.layers['log1p'].copy()
adata_mural.obs['ct_s_pos'] = 'Mural' + ' ' + adata_mural.obs['Vessel size category'].str[0].str.capitalize()
del adata_mural.layers


adata_comb = adata.concatenate(adata_mural)
# adata_comb.obs['Cell Subtype_no_cc'].replace({'Arterial EC':'PAEC','Venous EC':'PVEC','Pericyte':'Mural','Vascular smooth muscle':'Mural'},inplace=True)
# adata_comb.obs['Vessel size category'] = adata_comb.obs['Vessel size category'].str[0].str.capitalize()
# adata_comb.obs['Cell Subtype size'] = adata_comb.obs['Cell Subtype_no_cc'].astype(str) + ' ' + adata_comb.obs['Vessel size category'].astype(str) 




from gprofiler import GProfiler
def convert_mouse_to_human(adata):
    """Convert var_names in an AnnData object from mouse to human gene names using gProfiler."""
    gp = GProfiler(return_dataframe=True)

    # Convert mouse genes to human
    conversion_df = gp.orth(organism='mmusculus',query=adata.var_names.tolist(), target='hsapiens')

    # Extract mapping
    mouse_to_human = dict(zip(conversion_df['incoming'], conversion_df['name']))
    # Map `var_names`
    adata.var_names = adata.var_names.map(lambda x: mouse_to_human.get(x) if mouse_to_human.get(x) not in [None, 'N/A'] else x)
    adata.var_names = adata.var_names.str.upper() #Capitalize any unmapped genes
    # Remove duplicates (if any)
    adata.var_names_make_unique()
    return adata
    
convert_mouse_to_human(adata_comb)
li.mt.rank_aggregate(adata_comb,
                     groupby='ct_s_pos',
                     resource_name='consensus',
                     expr_prop=0.1,
                     use_raw=False,
                     verbose=True)
arts =  [
'PAEC L',

'PAEC M',
'PAEC S',
    'Cap1-Artery'
]
murs = [
'Mural L',
    'Mural M',
    'Mural S'
    
]

In [ ]:
df_plot = adata_comb.uns['liana_res'].copy()
df_plot

In [ ]:
source_target_pairs = [('PAEC L','Mural L'),
                          ('PAEC M','Mural M'),
                          ('PAEC S','Mural S'),
                          ('Cap1-Artery','Mural C'),]
ligand_receptor_pairs=[
    ('EFNB2','EPHB2'),
    ('EFNB2','EPHB4'),
    ('EFNB2','EPHB6'),
    ('EFNB2','EPHA3'),
    ('EFNA5','EPHB6'),
    ('EFNA5','EPHB2'),
    ('EFNA5','EPHA3'),
]
df_plot = adata_comb.uns['liana_res'].copy()
df_plot = df_plot[df_plot.apply(lambda r: (r["source"], r["target"]) in source_target_pairs, axis=1)]
df_plot = df_plot[df_plot.apply(lambda r: (r["ligand_complex"], r["receptor_complex"]) in ligand_receptor_pairs, axis=1)]
df_plot['-Log(Pval)'] = -np.log(df_plot['cellphone_pvals'])

source_target_order = [f'{x[0]} → {x[1]}' for x in source_target_pairs]
ligand_receptor_order = [f'{x[0].capitalize()} → {x[1].capitalize()}' for x in ligand_receptor_pairs]

df_plot["source_target"] = pd.Categorical(df_plot["source"].astype(str) + " → " + df_plot["target"].astype(str), 
                                          categories=source_target_order)
df_plot["channel"] = pd.Categorical(df_plot["ligand_complex"].astype(str).str.capitalize() + " → " + df_plot["receptor_complex"].astype(str).str.capitalize(),
                                    categories=ligand_receptor_order)


fig, ax = plt.subplots(1,1,figsize=(1,2.5))
sns.scatterplot(
        data=df_plot,
        y="channel",
        x="source_target",
        size='lr_means',
        hue='lr_logfc',
        sizes=(25,125),
        edgecolor="black",
        linewidth=0.5,
    palette='viridis',
    ax=ax

    )
ax.set_xticklabels(['L','M','S','C'], ha='right')
plt.legend(frameon=False)
plt.xlim((-0.5,3.5))
sns.move_legend(ax, "lower center", bbox_to_anchor=(1.55, 0))
# sns.despine(left=True, bottom=True,top=True,right=True)
ax.set_xlabel('')
ax.set_ylabel('')

fig.show()
fig.savefig(f'{figures}/dotplot_efn_signaling_artery_mural.png',dpi=300,bbox_inches='tight')

In [ ]:
order=['PAEC L','PAEC M','PAEC S','Cap1-Artery', 'Cap1-Vein','PVEC S','PVEC M', 'PVEC L']
vein_order=['Cap1-Vein','PVEC S','PVEC M', 'PVEC L']
sc.pl.DotPlot(adata,['Ncam1','Ncam2'],groupby='ct_s_pos',categories_order=order,
              standard_scale='var').style(cmap='viridis').savefig(f'{figures}/dotplot_ncam_all.png',dpi=300,bbox_inches='tight')
sc.pl.DotPlot(adata[adata.obs['ct_s_pos'].isin(vein_order)],['Ncam1','Ncam2'],groupby='ct_s_pos',categories_order=vein_order,
              standard_scale='var').style(cmap='viridis').savefig(f'{figures}/dotplot_ncam_vein.png',dpi=300,bbox_inches='tight')

In [ ]:
df_plot

In [ ]:
adata_comb.uns['liana_res'].columns

In [ ]:
source_target_pairs = [('PVEC L','PVEC L',),
                          ('PVEC M','PVEC M',),
                          ('PVEC S','PVEC S',),
                          ('Cap1-Vein','Cap1-Vein'),]
ligand_receptor_pairs=[
    ('NCAM1','NCAM2'),
]
df_plot = adata_comb.uns['liana_res'].copy()
df_plot['-Log(Pval)'] = -np.log10(df_plot['cellphone_pvals'])

df_plot = df_plot[df_plot.apply(lambda r: (r["source"], r["target"]) in source_target_pairs, axis=1)]
df_plot = df_plot[df_plot.apply(lambda r: (r["ligand_complex"], r["receptor_complex"]) in ligand_receptor_pairs, axis=1)]

source_target_order = [f'{x[0]} → {x[1]}' for x in source_target_pairs]
ligand_receptor_order = [f'{x[0].capitalize()} → {x[1].capitalize()}' for x in ligand_receptor_pairs]

df_plot["source_target"] = pd.Categorical(df_plot["source"].astype(str) + " → " + df_plot["target"].astype(str), 
                                          categories=source_target_order)
df_plot["channel"] = pd.Categorical(df_plot["ligand_complex"].astype(str).str.capitalize() + " → " + df_plot["receptor_complex"].astype(str).str.capitalize(),
                                    categories=ligand_receptor_order)


fig, ax = plt.subplots(1,1,figsize=(1,1))
sns.scatterplot(
        data=df_plot,
        y="channel",
        x="source_target",
        size='lr_logfc',
        hue='expr_prod',
        sizes=(50,200),
        edgecolor="black",
        linewidth=0.5,
    palette='viridis',
    ax=ax

    )
ax.set_xticklabels(['L','M','S',], ha='right')
plt.legend(frameon=False)
plt.xlim((-0.5,2.5))
sns.move_legend(ax, "lower center", bbox_to_anchor=(2, 0.05))
# sns.despine(left=True, bottom=True,top=True,right=True)
ax.set_xlabel('')
ax.set_ylabel('')

fig.show()

In [ ]:
p = li.pl.dotplot(adata = adata_comb,
              colour='lr_means',
              size='expr_prod',
              inverse_size=False, # we inverse sign since we want small p-values to have large sizes
              source_labels=arts,
              target_labels=murs,
              figure_size=(6, 4),
              # finally, since cpdbv2 suggests using a filter to FPs
              # we filter the pvals column to <= 0.05
              # ligand_complex=['JAG1','EFNA5','EFNB2','EFNB4','DKK2','ADAM17'],
              # receptor_complex=['EPHA1','EPHA3','EPHB2','EPHB4','EPHB6','NOTCH3','KREMEN','ERBB4'],
              ligand_complex=['EFNB2','EFNA5'],
              # receptor_complex=['EPHB6', 'EPHB4', 'EPHB3','EPHB2', 'EPHA3','EPHA1'],
                  size_range=(0.5,4.5),
              
              uns_key='liana_res' # uns_key to use, default is 'liana_res'
             )
from plotnine import theme, element_text
p=p+theme(
    text=element_text(size=10,colour='black', face='plain', family='Arial'),  # base setting
    axis_title_x=element_text(size=10, family='Arial'),
    axis_title_y=element_text(size=10, family='Arial'),
    axis_text_x=element_text(size=8, angle = 45,colour='black', face='plain',family='Arial'),
    axis_text_y=element_text(size=8, family='Arial'),
    legend_title=element_text(size=10, family='Arial'),
    legend_text=element_text(size=8, family='Arial'),
    plot_title=element_text(size=10, family='Arial'),
    plot_subtitle=element_text(size=10, family='Arial'),
    plot_caption=element_text(size=10, family='Arial'),
    strip_text_x=element_text(size=10, family='Arial'),
    strip_text_y=element_text(size=10, family='Arial'),
)
p.save(f'{figures}/dotplot_efn_signaling_artery.png',dpi=300)
p

In [ ]:
vens =  [
'PVEC L',

'PVEC M',
'PVEC S',
    'Cap1-Vein'
]
murs = [
'Mural L',
    'Mural M',
    'Mural S',
        'Mural C'
    
]
p = li.pl.dotplot(adata = adata_comb,
              colour='lr_means',
              size='expr_prod',
              inverse_size=False, # we inverse sign since we want small p-values to have large sizes
              source_labels=vens+murs,
              target_labels=vens+murs,
              figure_size=(10, 4),
              ligand_complex=['NCAM1','NCAM2'],
              receptor_complex=['NCAM1','NCAM2'],
                  size_range=(0.5,4.5),
              
              uns_key='liana_res' # uns_key to use, default is 'liana_res'
             )
from plotnine import theme, element_text
p=p+theme(
    text=element_text(size=10,colour='black', face='plain', family='Arial'),  # base setting
    axis_title_x=element_text(size=10, family='Arial'),
    axis_title_y=element_text(size=10, family='Arial'),
    axis_text_x=element_text(size=8, angle = 45,colour='black', face='plain',family='Arial'),
    axis_text_y=element_text(size=8, family='Arial'),
    legend_title=element_text(size=10, family='Arial'),
    legend_text=element_text(size=8, family='Arial'),
    plot_title=element_text(size=10, family='Arial'),
    plot_subtitle=element_text(size=10, family='Arial'),
    plot_caption=element_text(size=10, family='Arial'),
    strip_text_x=element_text(size=10, family='Arial'),
    strip_text_y=element_text(size=10, family='Arial'),
)
p.save(f'{figures}/dotplot_ccc_ncam.png',dpi=300)

In [ ]:

p = li.pl.dotplot(adata = adata_comb,
              colour='lr_means',
              size='expr_prod',
              inverse_size=False, # we inverse sign since we want small p-values to have large sizes
              # source_labels=vens,
              # target_labels=vens,
              figure_size=(10, 4),
              ligand_complex=['ADAM23'],
              # receptor_complex=['NCAM1','NCAM2'],
                  size_range=(0.5,4.5),
              
              uns_key='liana_res' # uns_key to use, default is 'liana_res'
             )
from plotnine import theme, element_text
p=p+theme(
    text=element_text(size=10,colour='black', face='plain', family='Arial'),  # base setting
    axis_title_x=element_text(size=10, family='Arial'),
    axis_title_y=element_text(size=10, family='Arial'),
    axis_text_x=element_text(size=8, angle = 45,colour='black', face='plain',family='Arial'),
    axis_text_y=element_text(size=8, family='Arial'),
    legend_title=element_text(size=10, family='Arial'),
    legend_text=element_text(size=8, family='Arial'),
    plot_title=element_text(size=10, family='Arial'),
    plot_subtitle=element_text(size=10, family='Arial'),
    plot_caption=element_text(size=10, family='Arial'),
    strip_text_x=element_text(size=10, family='Arial'),
    strip_text_y=element_text(size=10, family='Arial'),
)
p.save(f'{figures}/dotplot_ccc_adam23.png',dpi=300)